<a href="https://colab.research.google.com/github/alexconron/PDF_Processor_Notebook/blob/main/pdf_processor_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatización de Extracción de Datos de Documentos CNPJ y Exportación a Google Sheets

Este notebook contiene un script para automatizar la extracción de información específica (como CNPJ, nombre empresarial, dirección, etc.) de documentos PDF que contienen datos de registro de CNPJ, y para exportar los resultados a una hoja de cálculo de Google Sheets.

## Descripción General

El script utiliza expresiones regulares (regex) para escanear el texto de los documentos PDF y extraer campos de datos clave. Los resultados de la extracción se recopilan en un DataFrame de pandas y luego se escriben automáticamente en una hoja de cálculo de Google Sheets.

## Requisitos

*   **Google Colab con Google Drive montado:** Asegúrate de que tu cuenta de Google Drive esté conectada para acceder a los archivos PDF.
*   **Credenciales de Google configuradas:** Necesitas tener configuradas las credenciales para permitir que Colab acceda a tus hojas de cálculo de Google Sheets.
*   **Librerías Python:** Las librerías `PyPDF2` y `PyMuPDF` son necesarias para la extracción de texto de PDFs, `pandas` para la manipulación de datos y `gspread` junto con `google.auth` para la interacción con Google Sheets. Asegúrate de ejecutar la celda de instalación si es la primera vez que usas el notebook en una nueva sesión.

## Funciones Principales

*   `CNPJDocumentScanner`: Clase que contiene la lógica para extraer datos de los documentos PDF utilizando patrones regex.
*   `setup_google_services()`: Configura la conexión a los servicios de Google (Google Sheets).
*   `get_pdf_list_from_sheet(sheet_name, worksheet_index)`: Lee los nombres de los archivos PDF de una hoja de cálculo de Google Sheets especificada.
*   `write_results_to_sheet(df, sheet_name, worksheet_index)`: Escribe un DataFrame de pandas a una hoja de cálculo de Google Sheets.
*   `process_pdfs_from_sheet(sheet_name, pdf_dir, worksheet_index, output_sheet_name)`: La función principal que orquesta el proceso: lee la lista de PDFs de un sheet, escanea cada PDF en un directorio local (normalmente montado desde Google Drive), y escribe los resultados a otro sheet.
*   `manual_process_with_your_setup(sheet_name, pdf_dir)`: Una función alternativa para procesar PDFs de forma manual si prefieres no escribir directamente los resultados a un sheet de salida, sino obtener un DataFrame como resultado.

## Instrucciones de Uso

1.  **Montar Google Drive:** Asegúrate de que tu Google Drive esté montado en Colab para que el script pueda acceder a la carpeta que contiene tus archivos PDF.
2.  **Instalar Dependencias:** Ejecuta la celda de código que contiene `!pip install PyPDF2 PyMuPDF`. Recuerda que esto debe hacerse cada vez que inicies una nueva sesión.
3.  **Configurar Parámetros:** En la celda que define las variables `sheet_name`, `pdf_dir`, `worksheet_index` y `output_sheet`, actualiza los valores según tu caso:
    *   `sheet_name`: Nombre exacto de la hoja de cálculo de Google Sheets donde tienes la lista de nombres de archivos PDF.
    *   `pdf_dir`: Ruta a la carpeta en tu Google Drive (o sistema de archivos de Colab) donde se encuentran los archivos PDF a procesar.
    *   `worksheet_index`: Índice de la hoja dentro de la hoja de cálculo (`sheet_name`) que contiene la lista de nombres de archivos PDF (por defecto es 0, que corresponde a la primera hoja).
    *   `output_sheet`: Nombre de la hoja de cálculo de Google Sheets donde quieres que se escriban los resultados. Si la hoja no existe, se creará.
4.  **Ejecutar el Proceso:** Ejecuta la celda que llama a la función `process_pdfs_from_sheet` con los parámetros configurados.
5.  **Verificar Resultados:** Una vez que la ejecución finalice, los resultados se habrán escrito en la hoja de cálculo de Google Sheets especificada por `output_sheet`. También puedes inspeccionar el DataFrame `df_results` retornado por la función.

In [ ]:
# =============================================================================
# Importante, cada vez que se inicia el kernel (se vuelve a ingresar al archivo)
# SE DEBE DESCOMENTAR LA LINEA 5 QUITANDOLE EL '#'
# =============================================================================
# !pip install PyPDF2 PyMuPDF

In [ ]:
# =============================================================================
# INSTRUCCIONES DE USO
# =============================================================================

📋 FUNCIONES DISPONIBLES:
- quick_process_tk_171104() - Procesa tu caso específico
- manual_process_with_your_setup() - Usa tu configuración actual
- process_pdfs_from_sheet() - Función completa personalizable

🔧 REQUISITOS:
- Google Colab con Drive montado
- Credenciales de Google configuradas
- PyPDF2 y PyMuPDF instalados

💾 Los resultados se pueden:
- Exportar a Excel
- Escribir automáticamente a Google Sheets
- Usar como DataFrame de pandas

In [ ]:
import re
import pandas as pd
import PyPDF2
import fitz  # PyMuPDF
from pathlib import Path
import json
from typing import Dict, List, Optional, Union
import warnings
import gspread
from google.auth import default
from datetime import datetime
import os

warnings.filterwarnings('ignore')

In [ ]:
class CNPJDocumentScanner:
    """
    Clase para escanear documentos CNPJ y extraer información usando patrones regex
    Integrada con Google Sheets y Google Drive
    """

    def __init__(self):
        # # Patrones originales del usuario
        # self.user_patterns = {
        #     "NOME_EMPRESA": re.compile(r"Nome\s+da\s+Empresa\s*:\s*(.+)", re.I),
        #     "UF": re.compile(r"\bUF\s*:\s*([A-Z]{2})\b", re.I),
        #     "CNPJ": re.compile(r"\bCNPJ\s*:\s*([\d\./\-]{8,20})", re.I),
        #     "IE": re.compile(r"Inscri[cç][aã]o\s+Estadual\s*\(IE\)\s*:\s*([0-9A-Z\.\-\/]+)", re.I),
        #     "LOGRADOURO": re.compile(r"Logradouro\s*:\s*(.+)", re.I),
        #     "NRO": re.compile(r"\bNro\s*:\s*([0-9A-Z\-\/]+)", re.I),
        #     "COMPLEMENTO": re.compile(r"Complemento\s*:\s*(.+)", re.I),
        #     "BAIRRO": re.compile(r"Bairro\s*:\s*(.+)", re.I),
        #     "CEP": re.compile(r"\bCEP\s*:\s*([\d\.\-]{7,10})", re.I),
        #     "MUNICIPIO": re.compile(r"Munic[ií]pio\s*(?:IBGE)?\s*:\s*(?:\d+\s*-\s*)?(.+)", re.I),
        # }

        # Patrones mejorados para documentos CNPJ brasileños
        self.enhanced_patterns = {
            "CNPJ": re.compile(r"(?:NÚMERO\s+DE\s+INSCRIÇÃO|CNPJ)\s*\n([\d\./\-]{8,20})", re.I | re.MULTILINE),
            "DATA_ABERTURA": re.compile(r"DATA\s+DE\s+ABERTURA\s*\n(\d{2}\/\d{2}\/\d{4})", re.I | re.MULTILINE),
            "NOME_EMPRESARIAL": re.compile(r"(?:NOME\s+EMPRESARIAL|Nome\s+da\s+Empresa)\s*\n(.+?)(?:\n|$)", re.I | re.MULTILINE),
            "TITULO_ESTABELECIMENTO": re.compile(r"TÍTULO\s+DO\s+ESTABELECIMENTO\s*\(?NOME\s+DE\s+FANTASIA\)?\s*\n(.+?)(?:\n(?:[A-ZÁÉÍÓÚÂÊÔÇ]{2,}|PORTE|\d))", re.I | re.MULTILINE),
            "PORTE": re.compile(r"PORTE\s*\n(.+?)(?:\n|CÓDIGO)", re.I | re.MULTILINE),
            "ATIVIDADE_PRINCIPAL": re.compile(r"CÓDIGO\s+E\s+DESCRIÇÃO\s+DA\s+ATIVIDADE\s+ECONÔMICA\s+PRINCIPAL\s*\n(.+?)(?:\n|CÓDIGO)", re.I | re.MULTILINE | re.DOTALL),
            "NATUREZA_JURIDICA": re.compile(r"CÓDIGO\s+E\s+DESCRIÇÃO\s+DA\s+NATUREZA\s+JURÍDICA\s*\n(.+?)(?:\n|LOGRADOURO)", re.I | re.MULTILINE),
            "LOGRADOURO": re.compile(r"LOGRADOURO\s*\n(.+?)(?:\n|NÚMERO)", re.I | re.MULTILINE),
            "NUMERO": re.compile(r"NÚMERO\s*\n(.+?)(?:\n|COMPLEMENTO)", re.I | re.MULTILINE),
            "COMPLEMENTO": re.compile(r"COMPLEMENTO\s*\n(.+?)(?:\n|CEP)", re.I | re.MULTILINE),
            "CEP": re.compile(r"CEP\s*\n([\d\.\-]{7,10})", re.I | re.MULTILINE),
            "BAIRRO": re.compile(r"BAIRRO\/DISTRITO\s*\n(.+?)(?:\n|MUNICÍPIO)", re.I | re.MULTILINE),
            "MUNICIPIO": re.compile(r"MUNICÍPIO\s*\n(.+?)(?:\n|UF)", re.I | re.MULTILINE),
            "UF": re.compile(r"UF\s*\n([A-Z]{2})", re.I | re.MULTILINE),
            "EMAIL": re.compile(r"ENDEREÇO\s+ELETRÔNICO\s*\n(.+?)(?:\n|TELEFONE)", re.I | re.MULTILINE),
            "TELEFONE": re.compile(r"TELEFONE\s*\n(.+?)(?:\n|ENTE)", re.I | re.MULTILINE),
            "SITUACAO_CADASTRAL": re.compile(r"SITUAÇÃO\s+CADASTRAL\s*\n(.+?)(?:\n(?:DATA\s+DA\s+SITUAÇÃO|MOTIVO))", re.I | re.MULTILINE),
            "DATA_SITUACAO": re.compile(r"DATA\s+DA\s+SITUAÇÃO\s+CADASTRAL\s*\n(\d{2}\/\d{2}\/\d{4})", re.I | re.MULTILINE),
        }

        # Combinar todos los patrones
        # self.all_patterns = {**self.user_patterns, **self.enhanced_patterns}
        self.all_patterns = {**self.enhanced_patterns}

    def extract_text_from_pdf_pypdf2(self, file_path: Union[str, Path]) -> str:
        """Extrae texto de PDF usando PyPDF2"""
        try:
            with open(file_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                text = ""
                for page in reader.pages:
                    text += page.extract_text() + "\n"
                return text
        except Exception as e:
            print(f"Error con PyPDF2: {e}")
            return ""

    def extract_text_from_pdf_pymupdf(self, file_path: Union[str, Path]) -> str:
        """Extrae texto de PDF usando PyMuPDF (mejor para documentos complejos)"""
        try:
            doc = fitz.open(file_path)
            text = ""
            for page in doc:
                text += page.get_text() + "\n"
            doc.close()
            return text
        except Exception as e:
            print(f"Error con PyMuPDF: {e}")
            return ""

    def extract_text_from_pdf(self, file_path: Union[str, Path]) -> str:
        """Extrae texto de PDF intentando múltiples métodos"""
        # Primero intenta con PyMuPDF (mejor calidad)
        text = self.extract_text_from_pdf_pymupdf(file_path)
        if not text.strip():
            # Si falla, intenta con PyPDF2
            text = self.extract_text_from_pdf_pypdf2(file_path)
        return text

    def clean_extracted_text(self, text: str) -> str:
        """Limpia el texto extraído"""
        if not text:
            return ""

        # Normaliza espacios en blanco
        text = re.sub(r'\s+', ' ', text)
        # Preserva saltos de línea importantes
        text = re.sub(r'([A-ZÁÉÍÓÚÂÊÔÇ]{3,})\s+([A-ZÁÉÍÓÚÂÊÔÇ]{3,})', r'\1\n\2', text)
        return text.strip()

    def extract_data_from_text(self, text: str, use_enhanced: bool = True) -> Dict[str, str]:
        """Extrae datos del texto usando los patrones regex"""
        # Inicializar todos los campos como vacíos
        extracted_data = {key: "" for key in self.all_patterns.keys()}

        patterns_to_use = self.all_patterns if use_enhanced else self.user_patterns

        for field_name, pattern in patterns_to_use.items():
            matches = pattern.findall(text)
            if matches:
                value = matches[0].strip()
                value = re.sub(r'\*+', '', value).strip().upper()
                # Limpieza específica para CEP - eliminar puntos y normalizar
                if field_name == 'CEP' and value:
                    value = re.sub(r'\.', '', value)  # Eliminar puntos
                if value and value != '':
                    extracted_data[field_name] = value

        return extracted_data

    def scan_document(self, file_path: Union[str, Path]) -> Dict[str, str]:
        """Escanea un documento y extrae la información"""
        file_path = Path(file_path)

        if not file_path.exists():
            raise FileNotFoundError(f"El archivo {file_path} no existe")

        if file_path.suffix.lower() == '.pdf':
            text = self.extract_text_from_pdf(file_path)
        else:
            # Asume que es un archivo de texto
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()

        if not text.strip():
            raise ValueError("No se pudo extraer texto del documento")

        return self.extract_data_from_text(text)

In [ ]:
# =============================================================================
# FUNCIONES INTEGRADAS CON GOOGLE SHEETS Y DRIVE
# =============================================================================

def setup_google_services():
    """Configura los servicios de Google"""
    try:
        creds, _ = default()
        gc = gspread.authorize(creds)
        return gc
    except Exception as e:
        print(f"Error configurando servicios de Google: {e}")
        return None

def get_pdf_list_from_sheet(sheet_name: str, worksheet_index: int = 0):
    """Obtiene la lista de PDFs desde Google Sheets"""
    try:
        gc = setup_google_services()
        if not gc:
            return []

        worksheet = gc.open(sheet_name).get_worksheet(worksheet_index)
        pdfs = worksheet.get_all_values()[1:]  # Excluye headers
        pdfs = [pdf[0] for pdf in pdfs if pdf[0]]  # Solo nombres no vacíos

        print(f"📋 Se encontraron {len(pdfs)} archivos en el sheet '{sheet_name}'")
        return pdfs

    except Exception as e:
        print(f"Error obteniendo lista del sheet: {e}")
        return []

def process_pdfs_from_sheet(sheet_name: str, pdf_dir: Union[str, Path],
                           worksheet_index: int = 0,
                           output_sheet_name: str = None):
    """
    Función principal que integra todo:
    1. Lee la lista de PDFs desde Google Sheets
    2. Procesa cada PDF
    3. Opcionalmente escribe los resultados a otro sheet
    """

    print("🚀 Iniciando procesamiento de PDFs desde Google Sheets")
    print("=" * 60)

    # Configurar el escáner
    scanner = CNPJDocumentScanner()

    # Obtener lista de PDFs del sheet
    pdf_names = get_pdf_list_from_sheet(sheet_name, worksheet_index)

    if not pdf_names:
        print("❌ No se encontraron PDFs para procesar")
        return pd.DataFrame()

    # Configurar directorio
    pdf_dir = Path(pdf_dir)
    if not pdf_dir.exists():
        print(f"❌ El directorio {pdf_dir} no existe")
        return pd.DataFrame()

    # Procesar cada PDF
    results = []
    processed_count = 0
    error_count = 0

    for i, pdf_name in enumerate(pdf_names, 1):
        try:
            # Construir ruta completa
            pdf_path = pdf_dir / pdf_name

            # Verificar si el archivo existe
            if not pdf_path.exists():
                print(f"⚠️  [{i}/{len(pdf_names)}] Archivo no encontrado: {pdf_name}")
                results.append({
                    'archivo': pdf_name,
                    'status': 'ERROR - Archivo no encontrado',
                    'fecha_procesamiento': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                })
                error_count += 1
                continue

            # Procesar el PDF
            print(f"🔍 [{i}/{len(pdf_names)}] Procesando: {pdf_name}")

            extracted_data = scanner.scan_document(pdf_path)
            extracted_data['archivo'] = pdf_name
            extracted_data['status'] = 'EXITOSO'
            extracted_data['fecha_procesamento'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')


            results.append(extracted_data)
            processed_count += 1

            print(f"✅ [{i}/{len(pdf_names)}] Completado: {pdf_name}")

        except Exception as e:
            print(f"❌ [{i}/{len(pdf_names)}] Error procesando {pdf_name}: {str(e)}")
            results.append({
                'archivo': pdf_name,
                'status': f'ERROR - {str(e)}',
                'fecha_procesamiento': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })
            error_count += 1

    # Crear DataFrame con resultados
    df_results = pd.DataFrame(results)

    print("\n📊 RESUMEN DEL PROCESAMIENTO")
    print("=" * 40)
    print(f"Total de archivos: {len(pdf_names)}")
    print(f"Procesados exitosamente: {processed_count}")
    print(f"Errores: {error_count}")

    # Opcionalmente escribir a Google Sheets
    if output_sheet_name and not df_results.empty:
        try:
            write_results_to_sheet(df_results, output_sheet_name)
        except Exception as e:
            print(f"Error escribiendo resultados al sheet: {e}")

    return df_results

def write_results_to_sheet(df: pd.DataFrame, sheet_name: str, worksheet_index: int = 0):
    """Escribe los resultados a Google Sheets"""
    try:
        gc = setup_google_services()
        if not gc:
            return False

        # Intentar abrir el sheet existente o crear uno nuevo
        try:
            spreadsheet = gc.open(sheet_name)
        except gspread.SpreadsheetNotFound:
            spreadsheet = gc.create(sheet_name)
            print(f"📝 Creado nuevo sheet: {sheet_name}")

        worksheet = spreadsheet.get_worksheet(worksheet_index)

        # Limpiar el worksheet
        worksheet.clear()

        # Convertir todas las columnas a string
        df = df.astype(str)

        # Convertir infinidades a None para evitar el error de JSON
        df = df.replace(['inf', '-inf'], None)


        # Escribir headers
        headers = list(df.columns)
        worksheet.update('A1', [headers])

        # Escribir datos
        data = df.values.tolist()
        if data:
            worksheet.update('A2', data)

        print(f"✅ Resultados escritos en el sheet: {sheet_name}")
        return True

    except Exception as e:
        print(f"Error escribiendo al sheet: {e}")
        return False

In [ ]:
def manual_process_with_your_setup(sheet_name: str, pdf_dir: str):
    """
    Versión manual usando tu setup actual
    """
    # Configurar credenciales
    gc = setup_google_services()

    get_pdf_list_from_sheet(sheet_name, worksheet_index=0)

    # Obtener el sheet
    worksheet = gc.open(sheet_name).get_worksheet(0)


    # Obtener los links del sheet
    pdfs = worksheet.get_all_values()[1:]
    pdfs = [pdf[0] for pdf in pdfs]

    # Crear el escáner
    scanner = CNPJDocumentScanner()

    # Procesar PDFs
    results = []
    for pdf_name in pdfs:
        try:
            pdf_path = pdf_dir / pdf_name
            if pdf_path.exists():
                data = scanner.scan_document(pdf_path)
                data['archivo'] = pdf_name
                results.append(data)
                print(f"✅ Procesado: {pdf_name}")
            else:
                print(f"❌ No encontrado: {pdf_name}")
        except Exception as e:
            print(f"❌ Error con {pdf_name}: {e}")

    return pd.DataFrame(results)

🧰 Parametros:
- sheet_name = Nombre de la planilla donde se encuentran los PDFs-
- pdf_dir = Directorio donde se encuentran los PDFs.
- worksheet_index = hoja/sheet donde se encuentran los links de PDFs (por defecto se ve en la planilla 0).
- output_sheet = Salida de los datos escaneados

In [ ]:
sheet_name = "TK_171104_prueba" # NOMBRE DEL SHEET
pdf_dir = Path("/content/drive/MyDrive/TK 171104/")
worksheet_index = 0
output_sheet = "TK_171104_resultados"

process_pdfs_from_sheet(sheet_name=sheet_name, pdf_dir=pdf_dir,
                           worksheet_index=worksheet_index,
                           output_sheet_name=output_sheet)

🚀 Iniciando procesamiento de PDFs desde Google Sheets
📋 Se encontraron 32 archivos en el sheet 'TK_171104_prueba'
🔍 [1/32] Procesando: 47379565005073.RECEITA.pdf
✅ [1/32] Completado: 47379565005073.RECEITA.pdf
🔍 [2/32] Procesando: 47379565000942.RECEITA.pdf
✅ [2/32] Completado: 47379565000942.RECEITA.pdf
🔍 [3/32] Procesando: 47379565005405.RECEITA.pdf
✅ [3/32] Completado: 47379565005405.RECEITA.pdf
🔍 [4/32] Procesando: 47379565001329.RECEITA.pdf
✅ [4/32] Completado: 47379565001329.RECEITA.pdf
🔍 [5/32] Procesando: 47379565003372 - Receita Federal.pdf
✅ [5/32] Completado: 47379565003372 - Receita Federal.pdf
🔍 [6/32] Procesando: 47379565007874.RECEITA.pdf
✅ [6/32] Completado: 47379565007874.RECEITA.pdf
🔍 [7/32] Procesando: 47379565005235.receita.pdf
✅ [7/32] Completado: 47379565005235.receita.pdf
🔍 [8/32] Procesando: 47379565012100 - Receita Federal.pdf
✅ [8/32] Completado: 47379565012100 - Receita Federal.pdf
🔍 [9/32] Procesando: 47379565008412 - Receita Federal.pdf
✅ [9/32] Completado:

,CNPJ,DATA_ABERTURA,NOME_EMPRESARIAL,TITULO_ESTABELECIMENTO,PORTE,ATIVIDADE_PRINCIPAL,NATUREZA_JURIDICA,LOGRADOURO,NUMERO,COMPLEMENTO,...,MUNICIPIO,UF,EMAIL,TELEFONE,SITUACAO_CADASTRAL,DATA_SITUACAO,archivo,status,fecha_procesamento,fecha_procesamiento
0,47.379.565/0050-73,29/03/1995,PROXXI TECNOLOGIA LTDA.,,DEMAIS,47.51-2-01 - COMÉRCIO VAREJISTA ESPECIALIZADO ...,206-2 - SOCIEDADE EMPRESÁRIA LIMITADA,R WALDEMAR PEREIRA DA SILVEIRA,320,QUADRA3 LOTE 2,...,BAURU,SP,IBMEVOCE@BR.IBM.COM,(11) 2132-3053,ATIVA,03/11/2005,47379565005073.RECEITA.pdf,EXITOSO,2025-09-25 13:53:24,NaN
1,47.379.565/0009-42,16/02/1981,PROXXI TECNOLOGIA LTDA.,,DEMAIS,47.51-2-01 - COMÉRCIO VAREJISTA ESPECIALIZADO ...,206-2 - SOCIEDADE EMPRESÁRIA LIMITADA,V EXPRESSA DE CONTAGEM,3115,GALPAO4 - MODULO D ANEXO PARTE 2,...,CONTAGEM,MG,IBMEVOCE@BR.IBM.COM,(11) 2132-3053,ATIVA,03/11/2005,47379565000942.RECEITA.pdf,EXITOSO,2025-09-25 13:53:24,NaN
2,47.379.565/0054-05,05/01/1996,PROXXI TECNOLOGIA LTDA.,,DEMAIS,47.51-2-01 - COMÉRCIO VAREJISTA ESPECIALIZADO ...,206-2 - SOCIEDADE EMPRESÁRIA LIMITADA,AV CORONEL ESCOLASTICO,310,ANEXO PARTE 2,...,CUIABA,MT,IBMEVOCE@BR.IBM.COM,(11) 2132-3053,ATIVA,03/11/2005,47379565005405.RECEITA.pdf,EXITOSO,2025-09-25 13:53:24,NaN
3,47.379.565/0013-29,17/05/1983,PROXXI TECNOLOGIA LTDA.,,DEMAIS,47.51-2-01 - COMÉRCIO VAREJISTA ESPECIALIZADO ...,206-2 - SOCIEDADE EMPRESÁRIA LIMITADA,TV DA LIBERDADE,81,GALPAO 10 GALPAO 9 PARTE 2,...,SALVADOR,BA,IBMEVOCE@BR.IBM.COM,(11) 2132-3053,ATIVA,03/11/2005,47379565001329.RECEITA.pdf,EXITOSO,2025-09-25 13:53:25,NaN
4,47.379.565/0033-72,09/06/1986,PROXXI TECNOLOGIA LTDA.,,DEMAIS,47.51-2-01 - COMÉRCIO VAREJISTA ESPECIALIZADO ...,206-2 - SOCIEDADE EMPRESÁRIA LIMITADA,AV WALLACE SIMONSEN,1749,ANEXO PARTE 2,...,SAO BERNARDO DO CAMPO,SP,IBMEVOCE@BR.IBM.COM,(11) 2132-3053,ATIVA,03/11/2005,47379565003372 - Receita Federal.pdf,EXITOSO,2025-09-25 13:53:25,NaN
5,47.379.565/0078-74,29/11/2000,PROXXI TECNOLOGIA LTDA.,,DEMAIS,95.11-8-00 - REPARAÇÃO E MANUTENÇÃO DE COMPUTA...,206-2 - SOCIEDADE EMPRESÁRIA LIMITADA,R LAGOA SANTA,380,,...,SAO JOSE DOS CAMPOS,SP,IBMEVOCE@BR.IBM.COM,(11) 2132-3053,ATIVA,03/11/2005,47379565007874.RECEITA.pdf,EXITOSO,2025-09-25 13:53:25,NaN
6,47.379.565/0052-35,29/03/1995,PROXXI TECNOLOGIA LTDA.,,DEMAIS,47.51-2-01 - COMÉRCIO VAREJISTA ESPECIALIZADO ...,206-2 - SOCIEDADE EMPRESÁRIA LIMITADA,R AMBROGIO BISOGNI,709,,...,CAMPINAS,SP,IBMEVOCE@BR.IBM.COM,(11) 2132-3053,ATIVA,03/11/2005,47379565005235.receita.pdf,EXITOSO,2025-09-25 13:53:25,NaN
7,47.379.565/0121-00,30/01/2001,PROXXI TECNOLOGIA LTDA.,PROXXI TECNOLOGIA LTDA,DEMAIS,47.51-2-01 - COMÉRCIO VAREJISTA ESPECIALIZADO ...,206-2 - SOCIEDADE EMPRESÁRIA LIMITADA,AV DESEMBARGADOR MOREIRA,2800,LOJA-01 PARTE,...,FORTALEZA,CE,IBMEVOCE@BR.IBM.COM,(11) 2132-3053,ATIVA,03/11/2005,47379565012100 - Receita Federal.pdf,EXITOSO,2025-09-25 13:53:25,NaN
8,47.379.565/0084-12,28/12/2000,PROXXI TECNOLOGIA LTDA.,,DEMAIS,47.51-2-01 - COMÉRCIO VAREJISTA ESPECIALIZADO ...,206-2 - SOCIEDADE EMPRESÁRIA LIMITADA,AV PERIMETRAL ( VAM ),979,QUADRA33 LOTE 2,...,GOIANIA,GO,IBMEVOCE@BR.IBM.COM,(11) 2132-3053,ATIVA,03/11/2005,47379565008412 - Receita Federal.pdf,EXITOSO,2025-09-25 13:53:25,NaN
9,47.379.565/0016-71,11/02/1983,PROXXI TECNOLOGIA LTDA.,,DEMAIS,47.51-2-01 - COMÉRCIO VAREJISTA ESPECIALIZADO ...,206-2 - SOCIEDADE EMPRESÁRIA LIMITADA,ROD JORNALISTA FRANCISCO AGUIRRE PROENCA (SP-,S/N,KM 9 PREDIO 10 SETOR MM08 E MM12,...,HORTOLANDIA,SP,IBMEVOCE@BR.IBM.COM,(11) 2132-3053,ATIVA,03/11/2005,47379565001671 - Receita Federal.pdf,EXITOSO,2025-09-25 13:53:25,NaN


In [ ]:
# pdf_dir = Path("/content/drive/MyDrive/TK 171104/")
# output_sheet = "TK_171104_resultados"

# manual_process_with_your_setup(sheet_name="TK_171104_prueba", pdf_dir=pdf_dir)